<a href="https://colab.research.google.com/github/marcovzla/discobert/blob/master/discobert_feb7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install ipython tqdm requests boto3 regex click joblib nltk scikit-learn jupyter
!pip install torch torchvision transformers
!git clone https://github.com/marcovzla/discobert.git



     |████████████████████████████████| 481kB 6.9MB/s 
     |████████████████████████████████| 3.1MB 43.3MB/s 
     |████████████████████████████████| 1.0MB 39.9MB/s 
     |████████████████████████████████| 870kB 46.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=38d812ab66cb99c9b157fcd25be9946d0e1e340680273c957ac2adf7fe51f296
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
Cloning into 'discobert'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 148 (delta 92), reused 92 (delta 43), pack-reused 0
Receiving objects: 100% (148/148), 31.81 KiB | 3.53 MiB/s, done.
Resolving deltas: 100% (92/92), done.


## Data is mounted in GDrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Running the model... from the github dir

In [4]:
cd discobert


[Errno 2] No such file or directory: 'discobert'
/content/discobert


Pull the repo for latest code

In [5]:
!git pull origin master
# ls

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/marcovzla/discobert
 * branch            master     -> FETCH_HEAD
   1fb713a..ff2b909  master     -> origin/master
Updating 1fb713a..ff2b909
Fast-forward
 train.py | 89 +++++++++++++++++++++++++++++++++++++---------------------------
 1 file changed, 52 insertions(+), 37 deletions(-)


# Training code

In [0]:
import os
import torch
from torch.optim import Adam
from tqdm.autonotebook import tqdm
from model import DiscoBertModel
from rst import load_annotations, iter_spans_only

DATA="../drive/My Drive/discobert/data/"
train_dir = os.path.join(DATA, 'training_subset')
# train_dir = os.path.join(DATA, 'toy')
val_dir = os.path.join(DATA, 'validation')
# val_dir = os.path.join(DATA, 'toy')
model_dir = "../drive/My Drive/discobert/models/colab"
# model_dir = "../."
lr = 1e-1
num_epochs = 10
device = 'cuda'


def train(num_epochs, learning_rate, device, train_dir, val_dir, model_dir):
    with open(os.path.join(model_dir, "log.txt"), 'a') as logfile:

        torch.cuda.empty_cache()
        discobert = DiscoBertModel()
        discobert.set_device(device, init_weights=True)
        discobert.to(device)

        # setup the optimizer, loss, etc
        optimizer = Adam(params=discobert.parameters(), lr=learning_rate)

        # for each epoch
        for epoch_i in range(num_epochs):
            print(f'Beginning epoch {epoch_i}')
            print(f'Beginning epoch {epoch_i}', file=logfile)

            for annotation in tqdm(list(load_annotations(train_dir))):
                discobert.zero_grad()
                loss, pred_tree = discobert(annotation.edus, annotation.dis)
                loss.backward()
                optimizer.step()

            print(f'Finished epoch {epoch_i}')
            print(f'Finished epoch {epoch_i}', file=logfile)

            # save model
            epoch_model_dir = os.path.join(model_dir, f'discobert_{epoch_i}')
            if not os.path.exists(epoch_model_dir):
                os.makedirs(epoch_model_dir)
            discobert.save_pretrained(epoch_model_dir)
            # evaluate on validation
            # if discobert.device == 'cuda':
            #     torch.cuda.empty_cache()
            #     torch.cuda.synchronize()
            predict(val_dir, discobert, logfile)


def predict(data_dir, discobert, logfile):
    all_gold_nodes = []
    all_pred_nodes = []

    all_gold_spans = []
    all_pred_spans = []

    for annotation in tqdm(list(load_annotations(data_dir))):
        pred_tree = discobert(annotation.edus)[0]

        ann_gold_nodes = annotation.dis.get_nonterminals()
        ann_gold_spans = [f'{annotation.docid}_{x}' for x in list(iter_spans_only(ann_gold_nodes))]
        all_gold_nodes.extend(ann_gold_nodes)
        all_gold_spans.extend(ann_gold_spans)

        ann_pred_nodes = pred_tree.get_nonterminals()
        ann_pred_spans = [f'{annotation.docid}_{x}' for x in list(iter_spans_only(ann_pred_nodes))]
        all_pred_nodes.extend(ann_pred_nodes)
        all_pred_spans.extend(ann_pred_spans)

    p, r, f1 = eval(all_gold_spans, all_pred_spans)
    print(f'P:{p}\tR:{r}\tF1:{f1}')
    print(f'P:{p}\tR:{r}\tF1:{f1}', file=logfile)
    

def eval(gold, pred):
    print(gold[:20])
    print(len(gold))
    print("-"*70)
    print(pred[:20])
    print(len(pred))
    print("-"*70)

    TP, FP, FN = 0, 0, 0
    for g in gold:
        if g in pred:
            TP += 1
            #print("TP:", g)
        else:
            # print("FN:", g)
            FN += 1

    for p in pred:
        if p not in gold:
            FP += 1
            # print("FP:", p)

    print(f"TP: {TP}\tFN: {FN}\tFP: {FP}")
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * ((precision * recall) / (precision + recall))
    return precision, recall, f1

# train(num_epochs, lr, device, train_dir, val_dir, model_dir)

# Actually run the training

In [0]:
train(num_epochs, lr, device, train_dir, val_dir, model_dir)

Beginning epoch 0


Finished epoch 0


['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(0, 11)', 'wsj_1171.out_range(0, 2)', 'wsj_1171.out_range(2, 11)', 'wsj_1171.out_range(2, 9)', 'wsj_1171.out_range(2, 4)', 'wsj_1171.out_range(4, 9)', 'wsj_1171.out_range(4, 6)', 'wsj_1171.out_range(6, 9)', 'wsj_1171.out_range(7, 9)', 'wsj_1171.out_range(9, 11)', 'wsj_1171.out_range(11, 114)', 'wsj_1171.out_range(11, 35)', 'wsj_1171.out_range(11, 28)', 'wsj_1171.out_range(11, 21)', 'wsj_1171.out_range(11, 19)', 'wsj_1171.out_range(11, 16)', 'wsj_1171.out_range(11, 15)', 'wsj_1171.out_range(11, 13)', 'wsj_1171.out_range(13, 15)']
2468
----------------------------------------------------------------------
['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(0, 113)', 'wsj_1171.out_range(0, 112)', 'wsj_1171.out_range(0, 111)', 'wsj_1171.out_range(0, 110)', 'wsj_1171.out_range(0, 109)', 'wsj_1171.out_range(0, 108)', 'wsj_1171.out_range(0, 107)', 'wsj_1171.out_range(0, 106)', 'wsj_1171.out_range(0, 105)', 'wsj_1171.out_range(0, 104)', 'wsj_117

Finished epoch 1


['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(0, 11)', 'wsj_1171.out_range(0, 2)', 'wsj_1171.out_range(2, 11)', 'wsj_1171.out_range(2, 9)', 'wsj_1171.out_range(2, 4)', 'wsj_1171.out_range(4, 9)', 'wsj_1171.out_range(4, 6)', 'wsj_1171.out_range(6, 9)', 'wsj_1171.out_range(7, 9)', 'wsj_1171.out_range(9, 11)', 'wsj_1171.out_range(11, 114)', 'wsj_1171.out_range(11, 35)', 'wsj_1171.out_range(11, 28)', 'wsj_1171.out_range(11, 21)', 'wsj_1171.out_range(11, 19)', 'wsj_1171.out_range(11, 16)', 'wsj_1171.out_range(11, 15)', 'wsj_1171.out_range(11, 13)', 'wsj_1171.out_range(13, 15)']
2468
----------------------------------------------------------------------
['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(1, 114)', 'wsj_1171.out_range(2, 114)', 'wsj_1171.out_range(3, 114)', 'wsj_1171.out_range(4, 114)', 'wsj_1171.out_range(5, 114)', 'wsj_1171.out_range(6, 114)', 'wsj_1171.out_range(7, 114)', 'wsj_1171.out_range(8, 114)', 'wsj_1171.out_range(9, 114)', 'wsj_1171.out_range(10, 114)', 'wsj_11

...

['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(0, 11)', 'wsj_1171.out_range(0, 2)', 'wsj_1171.out_range(2, 11)', 'wsj_1171.out_range(2, 9)', 'wsj_1171.out_range(2, 4)', 'wsj_1171.out_range(4, 9)', 'wsj_1171.out_range(4, 6)', 'wsj_1171.out_range(6, 9)', 'wsj_1171.out_range(7, 9)', 'wsj_1171.out_range(9, 11)', 'wsj_1171.out_range(11, 114)', 'wsj_1171.out_range(11, 35)', 'wsj_1171.out_range(11, 28)', 'wsj_1171.out_range(11, 21)', 'wsj_1171.out_range(11, 19)', 'wsj_1171.out_range(11, 16)', 'wsj_1171.out_range(11, 15)', 'wsj_1171.out_range(11, 13)', 'wsj_1171.out_range(13, 15)']
2468
----------------------------------------------------------------------
['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(1, 114)', 'wsj_1171.out_range(2, 114)', 'wsj_1171.out_range(3, 114)', 'wsj_1171.out_range(4, 114)', 'wsj_1171.out_range(5, 114)', 'wsj_1171.out_range(6, 114)', 'wsj_1171.out_range(7, 114)', 'wsj_1171.out_range(8, 114)', 'wsj_1171.out_range(9, 114)', 'wsj_1171.out_range(10, 114)', 'wsj_11

Finished epoch 6


['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(0, 11)', 'wsj_1171.out_range(0, 2)', 'wsj_1171.out_range(2, 11)', 'wsj_1171.out_range(2, 9)', 'wsj_1171.out_range(2, 4)', 'wsj_1171.out_range(4, 9)', 'wsj_1171.out_range(4, 6)', 'wsj_1171.out_range(6, 9)', 'wsj_1171.out_range(7, 9)', 'wsj_1171.out_range(9, 11)', 'wsj_1171.out_range(11, 114)', 'wsj_1171.out_range(11, 35)', 'wsj_1171.out_range(11, 28)', 'wsj_1171.out_range(11, 21)', 'wsj_1171.out_range(11, 19)', 'wsj_1171.out_range(11, 16)', 'wsj_1171.out_range(11, 15)', 'wsj_1171.out_range(11, 13)', 'wsj_1171.out_range(13, 15)']
2468
----------------------------------------------------------------------
['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(1, 114)', 'wsj_1171.out_range(2, 114)', 'wsj_1171.out_range(3, 114)', 'wsj_1171.out_range(4, 114)', 'wsj_1171.out_range(5, 114)', 'wsj_1171.out_range(6, 114)', 'wsj_1171.out_range(7, 114)', 'wsj_1171.out_range(8, 114)', 'wsj_1171.out_range(9, 114)', 'wsj_1171.out_range(10, 114)', 'wsj_11

Finished epoch 7


['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(0, 11)', 'wsj_1171.out_range(0, 2)', 'wsj_1171.out_range(2, 11)', 'wsj_1171.out_range(2, 9)', 'wsj_1171.out_range(2, 4)', 'wsj_1171.out_range(4, 9)', 'wsj_1171.out_range(4, 6)', 'wsj_1171.out_range(6, 9)', 'wsj_1171.out_range(7, 9)', 'wsj_1171.out_range(9, 11)', 'wsj_1171.out_range(11, 114)', 'wsj_1171.out_range(11, 35)', 'wsj_1171.out_range(11, 28)', 'wsj_1171.out_range(11, 21)', 'wsj_1171.out_range(11, 19)', 'wsj_1171.out_range(11, 16)', 'wsj_1171.out_range(11, 15)', 'wsj_1171.out_range(11, 13)', 'wsj_1171.out_range(13, 15)']
2468
----------------------------------------------------------------------
['wsj_1171.out_range(0, 114)', 'wsj_1171.out_range(0, 64)', 'wsj_1171.out_range(0, 39)', 'wsj_1171.out_range(0, 25)', 'wsj_1171.out_range(0, 18)', 'wsj_1171.out_range(0, 14)', 'wsj_1171.out_range(0, 12)', 'wsj_1171.out_range(0, 11)', 'wsj_1171.out_range(0, 10)', 'wsj_1171.out_range(0, 9)', 'wsj_1171.out_range(0, 8)', 'wsj_1171.out_range(